# 01 时间动态分析：小时级情感演变

**研究重点**: Charlie Kirk暗杀事件后72小时的推文量与情感演变

**分析目标**:
1. 小时级推文量时间序列（特别是前24小时）
2. 情感随时间的演变（6大情感 x 72小时）
3. 叙事框架随时间的消长
4. 关键时间节点标注

In [1]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
project_root = Path('/workspace')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
print(f"✅ Python 路径已配置: {project_root}")

✅ Python 路径已配置: /workspace


## 步骤 1: 加载数据

In [2]:
import polars as pl
from pathlib import Path
from datetime import datetime, timezone

# 加载enriched数据（包含event_time_delta_hours）
df = pl.read_parquet("../parquet/tweets_enriched.parquet")
print(f"📊 数据加载完成: {df.height:,} 行, {df.width} 列")

# 加载内容分析数据（包含情感和叙事）
content_df = pl.read_parquet("../parquet/content_analysis.parquet")
print(f"📊 内容分析数据: {content_df.height:,} 行")

print(f"\n数据时间范围:")
print(f"  最早: {df['createdAt'].min()}")
print(f"  最晚: {df['createdAt'].max()}")
print(f"  跨度: {(df['createdAt'].max() - df['createdAt'].min()).total_seconds() / 3600:.1f} 小时")

📊 数据加载完成: 508,954 行, 25 列
📊 内容分析数据: 2,000 行

数据时间范围:
  最早: 2025-09-11 23:55:56+00:00
  最晚: 2025-09-13 00:12:32+00:00
  跨度: 24.3 小时


## 步骤 2: 构建小时级时间序列

In [3]:
# 添加小时维度列
df_hourly = df.with_columns([
    pl.col('createdAt').dt.truncate('1h').alias('hour'),
    pl.col('event_time_delta_hours').cast(pl.Int32).alias('hour_since_shooting')
])

# 按小时聚合推文量和互动量
hourly_counts = df_hourly.group_by('hour').agg([
    pl.len().alias('tweet_count'),
    pl.col('retweetCount').sum().alias('total_retweets'),
    pl.col('likeCount').sum().alias('total_likes'),
    pl.col('replyCount').sum().alias('total_replies'),
    (pl.col('retweetCount') + pl.col('likeCount') + pl.col('replyCount')).sum().alias('total_engagement'),
    pl.col('hour_since_shooting').first().alias('hours_since_event')
]).sort('hour')

print(f"\n⏰ 小时级时间序列:")
print(f"  总时间点: {hourly_counts.height} 小时")
print(f"\n前10小时数据:")
print(hourly_counts.head(10))


⏰ 小时级时间序列:
  总时间点: 26 小时

前10小时数据:
shape: (10, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ hour         ┆ tweet_count ┆ total_retwe ┆ total_likes ┆ total_repli ┆ total_engag ┆ hours_since │
│ ---          ┆ ---         ┆ ets         ┆ ---         ┆ es          ┆ ement       ┆ _event      │
│ datetime[μs, ┆ u32         ┆ ---         ┆ i64         ┆ ---         ┆ ---         ┆ ---         │
│ UTC]         ┆             ┆ i64         ┆             ┆ i64         ┆ i64         ┆ i32         │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 2025-09-11   ┆ 1542        ┆ 23116       ┆ 133996      ┆ 7536        ┆ 164648      ┆ 487743795   │
│ 23:00:00 UTC ┆             ┆             ┆             ┆             ┆             ┆             │
│ 2025-09-12   ┆ 22070       ┆ 516146      ┆ 3132666     ┆ 230449      ┆ 3879261     ┆ 487744795   │
│ 00:00:00 UTC ┆             ┆          

## 步骤 3: 情感随时间演变

In [4]:
# 为content_df添加小时列
content_hourly = content_df.with_columns([
    pl.col('createdAt').dt.truncate('1h').alias('hour')
])

# 按小时聚合情感分数
emotion_hourly = content_hourly.group_by('hour').agg([
    pl.len().alias('tweet_count'),
    pl.col('emotion_sadness').mean().alias('avg_sadness'),
    pl.col('emotion_anger').mean().alias('avg_anger'),
    pl.col('emotion_fear').mean().alias('avg_fear'),
    pl.col('emotion_surprise').mean().alias('avg_surprise'),
    pl.col('emotion_joy').mean().alias('avg_joy'),
    pl.col('emotion_love').mean().alias('avg_love'),
    # 主导情感分布
    (pl.col('primary_emotion') == 'sadness').sum().alias('sadness_count'),
    (pl.col('primary_emotion') == 'anger').sum().alias('anger_count'),
    (pl.col('primary_emotion') == 'fear').sum().alias('fear_count'),
]).sort('hour')

print(f"\n🎭 情感时间序列:")
print(f"  总时间点: {emotion_hourly.height} 小时")
print(f"\n情感演变数据（前5小时）:")
print(emotion_hourly.head(5))


🎭 情感时间序列:
  总时间点: 26 小时

情感演变数据（前5小时）:
shape: (5, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ hour      ┆ tweet_cou ┆ avg_sadne ┆ avg_anger ┆ … ┆ avg_love ┆ sadness_c ┆ anger_cou ┆ fear_coun │
│ ---       ┆ nt        ┆ ss        ┆ ---       ┆   ┆ ---      ┆ ount      ┆ nt        ┆ t         │
│ datetime[ ┆ ---       ┆ ---       ┆ f64       ┆   ┆ f64      ┆ ---       ┆ ---       ┆ ---       │
│ μs, UTC]  ┆ u32       ┆ f64       ┆           ┆   ┆          ┆ u32       ┆ u32       ┆ u32       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 2025-09-1 ┆ 3         ┆ 0.146014  ┆ 0.401129  ┆ … ┆ 0.0      ┆ 0         ┆ 1         ┆ 0         │
│ 1         ┆           ┆           ┆           ┆   ┆          ┆           ┆           ┆           │
│ 23:00:00  ┆           ┆           ┆           ┆   ┆          ┆           ┆           ┆           │
│ UTC       ┆           ┆           

## 步骤 4: 叙事框架随时间演变

In [5]:
# 按小时统计各叙事框架的分布
narrative_hourly = content_hourly.group_by(['hour', 'primary_narrative']).agg(
    pl.len().alias('count')
).sort(['hour', 'count'], descending=[False, True])

print(f"\n📖 叙事时间序列:")
print(f"  总记录数: {narrative_hourly.height}")

# 展示每小时的top3叙事
print(f"\n各小时top3叙事（前3小时）:")
for hour in emotion_hourly.head(3)['hour']:
    top_narratives = narrative_hourly.filter(pl.col('hour') == hour).head(3)
    print(f"\n  {hour}:")
    for row in top_narratives.iter_rows(named=True):
        print(f"    - {row['primary_narrative']}: {row['count']} 条")


📖 叙事时间序列:
  总记录数: 142

各小时top3叙事（前3小时）:

  2025-09-11 23:00:00+00:00:
    - political_violence: 2 条
    - free_speech: 1 条

  2025-09-12 00:00:00+00:00:
    - memorial: 48 条
    - political_violence: 38 条
    - none: 10 条

  2025-09-12 01:00:00+00:00:
    - political_violence: 37 条
    - memorial: 35 条
    - none: 16 条


## 步骤 5: 合并推文量与情感数据

In [6]:
# 合并推文量和情感数据（外连接，因为采样数据可能不覆盖所有小时）
hourly_combined = hourly_counts.join(
    emotion_hourly,
    on='hour',
    how='left',
    suffix='_emotion'
)

print(f"\n📊 综合时间序列数据:")
print(f"  总时间点: {hourly_combined.height} 小时")
print(f"  字段数: {hourly_combined.width}")
print(f"\n数据预览:")
print(hourly_combined.head(5))


📊 综合时间序列数据:
  总时间点: 26 小时
  字段数: 17

数据预览:
shape: (5, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ hour      ┆ tweet_cou ┆ total_ret ┆ total_lik ┆ … ┆ avg_love ┆ sadness_c ┆ anger_cou ┆ fear_coun │
│ ---       ┆ nt        ┆ weets     ┆ es        ┆   ┆ ---      ┆ ount      ┆ nt        ┆ t         │
│ datetime[ ┆ ---       ┆ ---       ┆ ---       ┆   ┆ f64      ┆ ---       ┆ ---       ┆ ---       │
│ μs, UTC]  ┆ u32       ┆ i64       ┆ i64       ┆   ┆          ┆ u32       ┆ u32       ┆ u32       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 2025-09-1 ┆ 1542      ┆ 23116     ┆ 133996    ┆ … ┆ 0.0      ┆ 0         ┆ 1         ┆ 0         │
│ 1         ┆           ┆           ┆           ┆   ┆          ┆           ┆           ┆           │
│ 23:00:00  ┆           ┆           ┆           ┆   ┆          ┆           ┆           ┆           │
│ UTC       ┆           ┆       

## 步骤 6: 标注关键时间节点

In [7]:
# 识别关键时间点
print("\n🎯 关键时间节点识别:")

# 1. 推文量高峰
peak_volume = hourly_combined.sort('tweet_count', descending=True).head(1)
print(f"\n📈 推文量高峰:")
print(f"  时间: {peak_volume['hour'][0]}")
print(f"  推文数: {peak_volume['tweet_count'][0]:,}")

# 2. 情感强度高峰（sadness, anger, fear）
if 'avg_sadness' in hourly_combined.columns:
    peak_sadness = hourly_combined.filter(pl.col('avg_sadness').is_not_null()).sort('avg_sadness', descending=True).head(1)
    peak_anger = hourly_combined.filter(pl.col('avg_anger').is_not_null()).sort('avg_anger', descending=True).head(1)
    
    if peak_sadness.height > 0:
        print(f"\n😢 悲伤情感高峰:")
        print(f"  时间: {peak_sadness['hour'][0]}")
        print(f"  悲伤强度: {peak_sadness['avg_sadness'][0]:.3f}")
    
    if peak_anger.height > 0:
        print(f"\n😡 愤怒情感高峰:")
        print(f"  时间: {peak_anger['hour'][0]}")
        print(f"  愤怒强度: {peak_anger['avg_anger'][0]:.3f}")

# 3. 第一个小时（初始反应）
first_hour = hourly_combined.head(1)
print(f"\n⏰ 第一个小时（初始反应）:")
print(f"  时间: {first_hour['hour'][0]}")
print(f"  推文数: {first_hour['tweet_count'][0]:,}")


🎯 关键时间节点识别:

📈 推文量高峰:
  时间: 2025-09-12 14:00:00+00:00
  推文数: 35,588

😢 悲伤情感高峰:
  时间: 2025-09-12 18:00:00+00:00
  悲伤强度: 0.211

😡 愤怒情感高峰:
  时间: 2025-09-11 23:00:00+00:00
  愤怒强度: 0.401

⏰ 第一个小时（初始反应）:
  时间: 2025-09-11 23:00:00+00:00
  推文数: 1,542


## 步骤 7: 保存时间序列数据

In [8]:
from src import io

# 保存小时级推文量数据
hourly_path = Path("../parquet/tweets_hourly.parquet")
io.materialize_parquet(hourly_combined.lazy(), hourly_path)
print(f"✅ 小时级时间序列已保存: {hourly_path}")

# 保存叙事时间序列
narrative_hourly_path = Path("../parquet/narrative_hourly.parquet")
io.materialize_parquet(narrative_hourly.lazy(), narrative_hourly_path)
print(f"✅ 叙事时间序列已保存: {narrative_hourly_path}")

print(f"\n📊 数据概览:")
print(f"  小时级时间点: {hourly_combined.height}")
print(f"  情感维度: 6")
print(f"  叙事框架: 6")

✅ 小时级时间序列已保存: parquet/tweets_hourly.parquet
✅ 叙事时间序列已保存: parquet/narrative_hourly.parquet

📊 数据概览:
  小时级时间点: 26
  情感维度: 6
  叙事框架: 6


## ✅ 时间序列分析完成！

**生成的核心数据**:
- `tweets_hourly.parquet`: 小时级推文量 + 情感演变数据
- `narrative_hourly.parquet`: 叙事框架的小时级分布

**下一步**: 构建Dashboard展示所有分析结果